# Super Mario Bros Reinforcement Learning Algorithms

## Part 1: Setup Mario

In [ ]:
# Import the game
import gym_super_mario_bros
# Import the controls
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [ ]:
# Setup game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# Simplifies the controls, takes out all actions that don't make sense
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [ ]:
# Random agent
# Create a flag - restart or not
done = True
# Loop through each frame in the game
for step in range(100000): 
    # Start the game to begin with 
    if done: 
        # Start the game
        env.reset()
    # Do random actions
    state, reward, done, info = env.step(env.action_space.sample())
    # Show the game on the screen
    env.render()
# Close the game
env.close()

In [ ]:
env.close()

## Preprocessing

In [ ]:
# Wrappers to simplify environment: grayscale
from gym.wrappers import GrayScaleObservation
# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Import Matplotlib to show the impact of frame stacking
from matplotlib import pyplot as plt